# Return of Control (ROC)을 사용하여 Agent 만들기

이 노트북에서는 함수 정의 및 Return of Control를 위한 신규 기능을 사용하여 Amazon Bedrock용 에이전트를 만들어 보겠습니다. 

HR agent를 예로 들어 보겠습니다. 이 에이전트를 사용하면 사용 가능한 휴가 일수를 확인하고 새로운 휴가를 요청할 수 있습니다. 로컬 함수를 사용하여 사용 가능한 휴가 일수를 확인하고 새 휴가를 예약하는 로직을 정의하겠습니다. 이 예제에는 API Schema 파일이나 람다 함수가 필요하지 않습니다.

함수 호출을 위한 제어 반환을 통해 개발자는 액션 스키마를 정의하고 에이전트가 액션을 호출할 때마다 제어를 다시 가져올 수 있습니다. 이를 통해 개발자는 이미 사용 가능한 람다 접근 방식 외에도 비즈니스 로직을 구현할 수 있는 더 많은 옵션을 제공합니다. 또한 개발자는 제어권 반환을 통해 오케스트레이션 흐름을 계속 유지하면서 시간이 오래 걸리는 작업을 백그라운드에서 실행(비동기 실행)할 수 있습니다. 예를 들어 사용자가 세 개의 서로 다른 비디오 파일을 인코딩하도록 요청하는 경우 에이전트는 세 번의 개별 encode_video API 호출을 수행해야 합니다. 이제 개발자는 제어권 반환을 사용하여 첫 번째 호출의 결과를 기다릴 필요 없이 세 개의 API를 동시에 호출하는 워크플로를 구축할 수 있습니다. 이 기능을 사용하면 개발자는 에이전트의 비동기 실행을 통해 효율성을 높이고 작업 실행 방식에 대한 유연성을 높이며 워크플로우 관리를 개선할 수 있습니다.

이 예제에서는 [SQLite](https://www.sqlite.org/) 데이터베이스를 사용하여 일부 직원 데이터를 생성하겠습니다.

<img src="./images/lab3-architecture.png" alt="Lab 3 - Create Agents with Return of Control" style="height: 500px; width:950px;"/>


시작하기 전에 최신 버전이 있는지 확인하기 위해 botocore 및 boto3 패키지를 업데이트해 보겠습니다.

In [2]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

이제 boto3 버전을 확인하여 올바른 버전이 설치되었는지 확인해 보겠습니다. 버전이 1.34.90보다 크거나 같아야 합니다.

In [3]:
import boto3
import json
import time
import uuid
import pprint
import logging
print(boto3.__version__)

1.34.121


In [4]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

[2024-06-07 13:26:18,647] p4466 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Next we can set some configuration variables for the agent

In [6]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '322537213286')

In [7]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-roc"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "휴일 예약을 위한 HR assistance를 제공하는 Agent"
agent_instruction = "너는 HR agent이고, 직원들이 인사 정책을 이해하고 휴가 시간을 관리할 수 있도록 지원합니다."
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "직원의 사용 가능한 휴가 일수를 확인하고 시스템에서 새 휴가를 예약하는 Actions"
agent_alias_name = f"{agent_name}-alias"

## Agent 만들기
이제 에이전트를 만들겠습니다. 그러기 위해서는 먼저 기반 모델 호출을 허용하는 에이전트 정책과 이와 관련된 정책으로 에이전트 IAM 역할을 만들어야 합니다. 이 에이전트가 Claude Sonnet 모델을 호출할 수 있도록 허용하겠습니다.

In [8]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [9]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '03c05eef-035d-4ee3-b170-3fcc5adabc8e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 13:32:38 GMT',
   'x-amzn-requestid': '03c05eef-035d-4ee3-b170-3fcc5adabc8e',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### agent 생성
필요한 IAM 역할이 만들어지면 Bedrock agent 클라이언트를 사용하여 새 agent를 만들 수 있습니다. 이를 위해 `create_agent` 함수를 사용합니다. 여기에는 agent 이름, 기본 foundation 모델 및 instructions이 필요합니다. agent 설명을 제공할 수도 있고, 생성된 agent는 아직 준비되지 않은 상태입니다. 에이전트를 준비한 다음 이를 사용하여 작업을 호출하고 다른 API를 사용하는 데 중점을 두겠습니다.

In [10]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': 'e620a693-7e4b-4a09-a908-dd15d57e7ecf',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 13:34:11 GMT',
   'content-type': 'application/json',
   'content-length': '683',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e620a693-7e4b-4a09-a908-dd15d57e7ecf',
   'x-amz-apigw-id': 'Y_7dJG8uPHcEu7Q=',
   'x-amzn-trace-id': 'Root=1-66630c53-2db4dc684ff539b05cb03ddd'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:322537213286:agent/7JDVSZB7LQ',
  'agentId': '7JDVSZB7LQ',
  'agentName': 'hr-assistant-function-roc',
  'agentResourceRoleArn': 'arn:aws:iam::322537213286:role/AmazonBedrockExecutionRoleForAgents_hr-assistant-function-roc',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 6, 7, 13, 34, 11, 925547, tzinfo=tzlocal()),
  'description': '휴일 예약을 위한 HR assistance를 제공하는 Agent',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeconds': 1800,
  'ins

이제 다음 단계에서 사용할 수 있도록 agent ID를 로컬 변수에 저장해 보겠습니다.

In [11]:
agent_id = response['agent']['agentId']
agent_id

'7JDVSZB7LQ'

## Agent Action Group 생성
이제 Agent Action Group을 만들어 보겠습니다. 이 기능은 [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) 함수에서 제공합니다. 아직 agent version이나 alias을 만들지 않았으므로 `DRAFT`를 agent version으로 사용하겠습니다. agent에게 Action Group 기능을 알리기 위해 Action Group의 기능이 포함된 Action Group 설명을 제공합니다.

이 예에서는 `functionSchema`를 사용하여 Action Group 기능을 제공합니다. `APISchema`를 제공할 수도 있습니다. 노트북 [02-create-agent-with-api-schema.ipynb](02-create-agent-with-api-schema/02-create-agent-with-api-schema.ipynb)에 그 예가 나와 있습니다.

함수 스키마를 사용하여 함수를 정의하려면 각 함수에 대한 `name`, `description`와 `parameters`를 제공해야 합니다.

<img src="./images/HR_DB.png" alt="DB table schema" style="height: 400px; width:800px;"/>

In [12]:
agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': '특정 직원이 사용할 수 있는 휴가 수 가져오기',
        'parameters': {
            "employee_id": {
                "description": "사용 가능한 휴가를 얻기 위한 직원의 ID",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'reserve_vacation_time',
        'description': '특정 직원을 위한 휴가 시간 예약',
        'parameters': {
            "employee_id": {
                "description": "휴가를 예약할 직원의 ID",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "휴가 기간의 시작 날짜",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "휴가 기간의 종료 날짜",
                "required": True,
                "type": "string"
            }
        }
    },
]

여기서는 `RETURN_CONTROL`의 `customContrl` 실행자를 사용하여 action group을 만듭니다. 이렇게 하면 에이전트가 함수를 실행하는 대신 적절한 함수와 매개변수를 반환해야 한다는 것을 알 수 있습니다. 그러면 클라이언트 애플리케이션이 함수를 실행할 책임이 있습니다.

In [13]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [14]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': 'ca2e16aa-f641-4928-b8de-6aeb051fbb31',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 13:41:08 GMT',
   'content-type': 'application/json',
   'content-length': '1116',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ca2e16aa-f641-4928-b8de-6aeb051fbb31',
   'x-amz-apigw-id': 'Y_8eKGGRPHcERzg=',
   'x-amzn-trace-id': 'Root=1-66630df3-117b80d4123baca81bfa32be'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'customControl': 'RETURN_CONTROL'},
  'actionGroupId': '30JLR0TGUV',
  'actionGroupName': 'VacationsActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': '7JDVSZB7LQ',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 6, 7, 13, 41, 8, 216167, tzinfo=tzlocal()),
  'description': '직원의 사용 가능한 휴가 일수를 확인하고 시스템에서 새 휴가를 예약하는 Actions',
  'functionSchema': {'functions': [{'description': '특정 직원이 사용할 수 있는 휴가 수 가져오기',
     'name': 'get_available_vacations_days',
     'parameters': {'empl

## Agent 준비하기

내부 테스트에 사용할 수 있는 Agent의 DRAFT 버전을 만들어 보겠습니다.


In [15]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': '767534bd-f5a4-439c-86fd-ceb85a074fd1', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 13:41:55 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '767534bd-f5a4-439c-86fd-ceb85a074fd1', 'x-amz-apigw-id': 'Y_8lmGXTvHcEJug=', 'x-amzn-trace-id': 'Root=1-66630e23-4b241d6d5098821744671f7e'}, 'RetryAttempts': 0}, 'agentId': '7JDVSZB7LQ', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 6, 7, 13, 41, 55, 629276, tzinfo=tzlocal())}


## Agent 호출

이제 에이전트를 만들었으니 `bedrock-agent-runtime` 클라이언트를 사용하여 이 에이전트를 호출하고 몇 가지 작업을 수행해 보겠습니다.

In [16]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

In [19]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False
# Pause to make sure agent alias is ready
# time.sleep(30)

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="직원 1은 얼마나 많은 휴가를 사용할 수 있나요?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-06-07 13:53:54,125] p4466 {1167014641.py:18} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 13:53:54 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '60b37df6-24d5-11ef-a3f7-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'fe69640f-c076-41cb-9ea5-8115c7bd4d1e'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'fe69640f-c076-41cb-9ea5-8115c7bd4d1e',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f68129c2800>,
 'contentType': 'application/json',
 'sessionId': '60b37df6-24d5-11ef-a3f7-0a86a609e469'}


In [20]:
%%time
event_stream = agentResponse['completion']
for event in event_stream:
    if 'returnControl' in event:
        pprint.pp(event)

{'returnControl': {'invocationId': '8fa595c7-1c10-4450-9337-314d69758371',
                   'invocationInputs': [{'functionInvocationInput': {'actionGroup': 'VacationsActionGroup',
                                                                     'function': 'get_available_vacations_days',
                                                                     'parameters': [{'name': 'employee_id',
                                                                                     'type': 'integer',
                                                                                     'value': '1'}]}}]}}
CPU times: user 2.39 ms, sys: 0 ns, total: 2.39 ms
Wall time: 1.53 s


## 함수 구현 정의하기

이제 employee_id에 대한 휴가 정보를 가져오고 start_date와 end_dates 사이에 employee_id의 휴가를 예약하는 함수를 구현해 보겠습니다.

이를 위해 먼저 생성된 데이터로 SQLite 데이터베이스를 생성합니다.

In [21]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

    # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year - 3, -1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days - days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y-%m-%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) - date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()

생성된 파일 `employee_database.db`에 대해 `get_available_vacations_days` 및 `reserve_vacation_time`을 구현하여 쿼리 결과를 제공합니다.

In [22]:
def employee_db(employee_id):
    import pandas as pd
    import sqlite3

    # Read sqlite query results into a pandas DataFrame
    conn = sqlite3.connect('employee_database.db')
    df = pd.read_sql_query(f"SELECT a.employee_id, \
                           a.employee_name, \
                           a.employee_job_title, \
                           a.employee_start_date, \
                           a.employee_employment_status, \
                           b.year, \
                           b.employee_total_vacation_days, \
                           b.employee_vacation_days_taken, \
                           b.employee_vacation_days_available, \
                           c.vacation_start_date, \
                           c.vacation_end_date, \
                           c.vacation_days_taken \
                           from employees a, vacations b, planned_vacations c \
                           where a.employee_id=b.employee_id and b.employee_id=c.employee_id and a.employee_id={employee_id}", conn)
    conn.close()
    return df

In [24]:
import sqlite3

def get_available_vacations_days(employee_id):
    # Connect to the SQLite database
    conn = sqlite3.connect('employee_database.db')
    c = conn.cursor()

    if employee_id:

        # Fetch the available vacation days for the employee
        c.execute("""
            SELECT employee_vacation_days_available
            FROM vacations
            WHERE employee_id = ?
            ORDER BY year DESC
            LIMIT 1
        """, (employee_id,))

        available_vacation_days = c.fetchone()

        if available_vacation_days:
            available_vacation_days = available_vacation_days[0]  # Unpack the tuple
            print(f"Available vacation days for employed_id {employee_id}: {available_vacation_days}")
            conn.close()
            return available_vacation_days
        else:
            return_msg = f"No vacation data found for employed_id {employee_id}"
            print(return_msg)
            conn.close()
            return return_msg
    else:
        conn.close()
        raise Exception(f"No employeed id provided")
    
    
def reserve_vacation_time(employee_id, start_date, end_date):
    # Connect to the SQLite database
    conn = sqlite3.connect('employee_database.db')
    c = conn.cursor()
    try:
        # Calculate the number of vacation days
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        vacation_days = (end_date - start_date).days + 1

        # Get the current year
        current_year = start_date.year

        # Check if the employee exists
        c.execute("SELECT * FROM employees WHERE employee_id = ?", (employee_id,))
        employee = c.fetchone()
        if employee is None:
            return_msg = f"Employee with ID {employee_id} does not exist."
            print(return_msg)
            conn.close()
            return return_msg

        # Check if the vacation days are available for the employee in the current year
        c.execute("SELECT employee_vacation_days_available FROM vacations WHERE employee_id = ? AND year = ?", (employee_id, current_year))
        available_days = c.fetchone()
        if available_days is None or available_days[0] < vacation_days:
            return_msg = f"Employee with ID {employee_id} does not have enough vacation days available for the requested period."
            print(return_msg)
            conn.close()
            return return_msg

        # Insert the new vacation into the planned_vacations table
        c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, vacation_days))

        # Update the vacations table with the new vacation days taken
        c.execute("UPDATE vacations SET employee_vacation_days_taken = employee_vacation_days_taken + ?, employee_vacation_days_available = employee_vacation_days_available - ? WHERE employee_id = ? AND year = ?", (vacation_days, vacation_days, employee_id, current_year))

        conn.commit()
        print(f"Vacation booked successfully for employee with ID {employee_id} from {start_date} to {end_date}.")
        # Close the database connection
        conn.close()
        return f"Vacation booked successfully for employee with ID {employee_id} from {start_date} to {end_date}."
    except Exception as e:
        conn.rollback()
        # Close the database connection
        conn.close()
        raise Exception(f"Error occurred: {e}")

In [25]:
print(event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["function"])

get_available_vacations_days


이제 agent가 제공한 매개 변수를 사용하여 get_available_vacations_days 함수를 호출할 수 있습니다.

In [26]:
available_vacation_days = None
for invocationInput in event["returnControl"]["invocationInputs"]:
    function_to_call = invocationInput["functionInvocationInput"]["function"]
    if function_to_call == "get_available_vacations_days":
        employee_id = None
        for param in invocationInput["functionInvocationInput"]["parameters"]:
            if param["name"] == "employee_id":
                employee_id = param["value"]
        if employee_id:
            available_vacation_days = get_available_vacations_days(employee_id)
available_vacation_days

Available vacation days for employed_id 1: 12


12

# 함수 결과를 사용하여 에이전트 호출하기
마지막으로 함수 결과를 매개변수로 전달하여 에이전트를 호출해야 합니다. 이를 통해 최종 응답을 생성하는 데 에이전트를 사용할 수 있습니다.

In [27]:
raw_response_with_roc = bedrock_agent_runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    sessionState={
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
                'functionResult': {
                    'actionGroup': event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["actionGroup"],
                    'function': event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["function"],
                    'responseBody': {
                        "TEXT": {
                            'body': "available_vacation_days: "+str(available_vacation_days)
                        }
                    }
                }
        }]}
)

In [28]:
print(raw_response_with_roc)

{'ResponseMetadata': {'RequestId': '2ff0fcc0-3f96-40a1-a9bd-74de1b5a2198', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 14:19:31 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '2ff0fcc0-3f96-40a1-a9bd-74de1b5a2198', 'x-amz-bedrock-agent-session-id': '60b37df6-24d5-11ef-a3f7-0a86a609e469', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '60b37df6-24d5-11ef-a3f7-0a86a609e469', 'completion': <botocore.eventstream.EventStream object at 0x7f67efda41f0>}


In [29]:
%%time
event_stream = raw_response_with_roc['completion']
for event in event_stream:
    print(event)

{'chunk': {'bytes': b'\xec\xa7\x81\xec\x9b\x90 1\xec\x9d\x80 12\xec\x9d\xbc\xec\x9d\x98 \xed\x9c\xb4\xea\xb0\x80\xeb\xa5\xbc \xec\x82\xac\xec\x9a\xa9\xed\x95\xa0 \xec\x88\x98 \xec\x9e\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4.'}}
CPU times: user 1.79 ms, sys: 0 ns, total: 1.79 ms
Wall time: 876 ms


### 사용 가능한 함수로 해결할 수 없는 입력 텍스트를 ROC가 어떻게 처리하는지 보여주세요.

In [30]:
def simple_agent_roc_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    logger.info(pprint.pprint(agentResponse))
    event_stream = agentResponse['completion']
    try:
         for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            elif 'returnControl' in event:
                pprint.pp(event)
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)


먼저 agent 및 사용 가능한 기능과 관련이 없는 요청을 표시합니다.

In [31]:
simple_agent_roc_invoke("미국의 대통령은 누구인가요?", agent_id, agent_alias_id, session_id)


[2024-06-07 14:21:40,495] p4466 {492633146.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 14:21:40 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '60b37df6-24d5-11ef-a3f7-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '8d93b93a-f2ec-4367-9ed9-42426dde5c63'},
                      'HTTPStatusCode': 200,
                      'RequestId': '8d93b93a-f2ec-4367-9ed9-42426dde5c63',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f67efd33340>,
 'contentType': 'application/json',
 'sessionId': '60b37df6-24d5-11ef-a3f7-0a86a609e469'}


[2024-06-07 14:21:42,745] p4466 {492633146.py:16} INFO - Final answer ->
죄송합니다. 저는 휴가 정책과 관련된 질문에만 답변할 수 있습니다. 대통령에 대한 질문은 답변할 수 없습니다.


이번에는 불충분한 매개변수가 제공되었을 때 어떤 일이 발생하는지 보여줍니다. 이 경우 휴가 시작 날짜가 없습니다.

In [32]:
simple_agent_roc_invoke("직원 2의 휴가를 2일 예약해 주세요.", agent_id, agent_alias_id, session_id)

[2024-06-07 14:22:51,010] p4466 {492633146.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 14:22:51 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '60b37df6-24d5-11ef-a3f7-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '1d10fb26-6f62-40ed-a4a1-07748d2d58ae'},
                      'HTTPStatusCode': 200,
                      'RequestId': '1d10fb26-6f62-40ed-a4a1-07748d2d58ae',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f67efd32f50>,
 'contentType': 'application/json',
 'sessionId': '60b37df6-24d5-11ef-a3f7-0a86a609e469'}


[2024-06-07 14:22:56,333] p4466 {492633146.py:16} INFO - Final answer ->
직원 2의 휴가를 예약하려면 시작 날짜와 종료 날짜가 필요합니다. 예를 들어 "2023년 6월 1일부터 2023년 6월 2일까지" 와 같은 형식으로 날짜를 제공해 주시기 바랍니다.


## Clean up (optional)

다음 단계는 선택 사항이며 에이전트를 삭제하는 방법을 보여줍니다. 에이전트를 삭제하려면 다음과 같이 해야 합니다:

1. 작업 그룹을 업데이트하여 비활성화 하기
2. agent action group 삭제하기
4. agent 삭제
5. 생성된 IAM 역할 및 정책 삭제하기


In [33]:
# This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [34]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [35]:
# Delete IAM Roles and policies

for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
for role_name in [agent_role_name]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)


## Conclusion
We have now experimented with using boto3 SDK to create, invoke and delete an agent created using function definitions.

## Take aways
Adapt this notebook to create new agents using function definitions for your application

## Thank You